In [1]:

import streamlit as st
import pandas as pd
import folium
from streamlit_folium import folium_static
import joblib
import os
main_dir = os.getcwd().rsplit("\\", 2)[0]

2025-08-28 23:17:32.619 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
# Load data and models
df = pd.read_csv(f"{main_dir}/data/processed/cleaned_data_final.csv")

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='mixed')
prophet_model = joblib.load(f"{main_dir}/models/prophet_wind.pkl")

In [ ]:
# Streamlit app
st.title("Renewable Energy Predictive Maintenance Dashboard")
energy_source = st.selectbox("Select Energy Source", ['wind', 'solar'])
date_range = st.date_input("Select Date Range", [df['date'].min(), df['date'].max()])
# Filter data
filtered_df = df[(df['energy_source'] == energy_source) & 
                 (df['date'].dt.date >= date_range[0]) & 
                 (df['date'].dt.date <= date_range[1])]
# Time series plot
st.line_chart(filtered_df.set_index('date')['power_MW'])


In [ ]:
# Geospatial map
m = folium.Map(location=[31.9686, -99.9018], zoom_start=6)
for idx, row in filtered_df.drop_duplicates(['latitude', 'longitude']).iterrows():
    folium.CircleMarker(
        [row['latitude'], row['longitude']],
        radius=row['output_efficiency'] * 10,
        popup=f"{row['plantname']} - {row['energy_source']}",
        color='blue' if row['energy_source'] == 'wind' else 'orange',
        fill=True
    ).add_to(m)
folium_static(m)